# Ensemble Method

Ensemble methods in machine learning refer to techniques that combine the predictions of multiple individual models (often called base or weak learners) to produce a stronger model.

The idea behind ensemble methods is to leverage the wisdom of crowds, where combining multiple weak models can often lead to better predictive performance than any individual model on its own.

## Decision Tree

We've already covered what is a decision tree. It can be used as a classifier or as a regressor. As a regressor the output in the mean of all samples in the leaf.



<div>
<img src="files/decision_tree_2.png" width="60%" align='center'/> </div>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris

iris = load_iris()
df = pd.DataFrame(data=np.c_[iris['data'], iris['target']],
                  columns= iris['feature_names'] + ['target'])
df.drop(columns=['sepal length (cm)', 'sepal width (cm)'], inplace=True) # Let's simplify the dataset
df

In [ ]:
sns.scatterplot(data=df, x="petal length (cm)", y="petal width (cm)", hue="target", palette="viridis", s=70);

In [ ]:
X = df.drop(columns=['target']).values
y = df['target'].values.astype(int)

In [ ]:
# Instantiate and train model
from sklearn.tree import DecisionTreeClassifier
max_depth = 2
tree_clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42)
tree_clf.fit(X,y)

In [ ]:
from sklearn import tree
print(tree.export_text(tree_clf))

In [ ]:
fig = plt.figure(figsize=(10,5))
tree.plot_tree(tree_clf,
               feature_names=df.drop(columns='target').columns,
               max_depth=max_depth,
               filled=True)

plt.show()

- The first "box" is called the "root node"
- The next ones are the "internal nodes".
- The ones with color are the "leaves" (prediction).

# Gini coefficient

Also known as the Gini impurity, is a measure of impurity or disorder in a set of data points. In a Decision Tree context, the Gini index measures the ability of each feature to separate the data. Lower values of Gini impurity indicate less impurity and better splits.

$Gini = 1 - \sum_{i=1}^{n} (p_i)^2$

$p_i$ being the ratio between the observations in a class and $i$ the total number of observations remaining at a given node.

In [ ]:
# Let's compute the gini score for the root node
1 - ((50 / 150)**2 + (50 / 150)**2 + (50 / 150)**2)

In [ ]:
# Let's compute the gini score for the green box
1 - ((0 / 54)**2 + (49 / 54)**2 + (5 / 54)**2)

### How Do We "Grow" a Tree?

The tree's structure is defined through the following steps:

1. Start at root node, which contains your entire dataset
1. Try various combinations of **(feature, threshold)** combos; each would split your dataset into 2 child nodes
1. For each combination, compute the weighted average Gini index of both child nodes (weighted by number of instances)
1. Select the (feature, threshold) combo yielding the lowest index (i.e. the "purest child nodes")
1. Split the dataset in two using this rule
1. Repeat step 2 for both subsets
1. Stop when no feature improves node impurity (at what risk?)

So to find the root node, the algorithm tried many different combinations with our two features and different thresold, trying to lower the mean of the gini score of the two next leaves. That's how it found "pental lenght (cm)" and <= 2.45.

In [ ]:
#pd.DataFrame(columns=X.columns, data=[[4, 1]]) # if X is a dataframe, you need to also provide a df

tree_clf.predict([[4, 1]]) # 'petal length (cm)' and 'petal width (cm)'

In [ ]:
tree_clf.predict_proba([[4, 1]]) # Not a true "probability", it's the ratio of classes in the node [0, 49, 5] for 54 samples

This is not a score, so it's not easily interpretable.

In [ ]:
from mlxtend.plotting import plot_decision_regions
plot_decision_regions(X, y, clf=tree_clf);

- The ```[4, 1]``` prediction is classified as a class 1.
- The 90% previsously seen is the ratio of class 1 (49 / 54) on total sample in that area.
- All separations are orthogonal to the features.

### Features importance

Using a decision tree can help to decide which features are important. The Gini importance of a feature is calculated by summing the Gini impurity decrease (or the weighted impurity decrease) across all the nodes of the tree where the feature is used for splitting.

- For each feature, the decision tree algorithm measures the decrease in Gini impurity that results from splitting on that feature.
- The weighted impurity decrease across all nodes where the feature is used for splitting is then calculated.
- Finally, the feature importances are normalized so that they sum up to 1.

In [ ]:
tree_clf.feature_importances_

### Decision trees VS other models

<div>
<img src="files/classifier_comparaison.png" width="100%" source='https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html' align='center'/> </div>

[Sklearn Website](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html)

### Controlling Overfitting

In order to have good results, decision trees must be tuned because default parameters will almost certainly overfit. You can control :

- Split with the ```min_samples_split``` parameter.
- Leaves with the ```min_samples_leaf``` parameter.
- Tree depth with ```the max_depth``` parameter.

### Decision Trees : Pros and Cons

**Advantages**

- No scaling necessary
- Resistant to outliers
- Intuitive and easy to interpret
- Allows for feature selection (see Gini-based feature_importance_)
- Non-linear modeling

**Disadvantages**

- High variance (i.e. a small change in the data causes a big change in the tree's structure)
- Long training time if grown up to (large) max depth.
- Splits data "orthogonally" to feature directions
- Use PCA upfront to "orient" data (see "unsupervised learning")

## Random Forests

To understand what are random forests, let's first see a new concept : **bagging**.

### Bagging (**B**ootstrap **Agg**regat**ing**)

#### Bootstrapping

Given a dataset of size $N$, a new dataset of the same size is created by randomly sampling $N$ examples from the original dataset with replacement. This means that each example in the original dataset can be sampled multiple times in the new dataset, or not at all.

#### Aggregating

This means we will train different models on each one of those new datasets and then combine them together. Some algorithms, like the random forest, also drop some features during the training process.

<div>
<img src="files/bagging.svg" width="75%" source='https://en.wikipedia.org/wiki/Bootstrap_aggregating' align='center'/> </div>

- Bagging is also called a "a parallel ensemble method" and aims to reduce variance.
- Each version of the model is called a **weak learner** (because each model is not very good on its own).

<div>
<img src="files/decision_tree_vs_random_forest.png" width="75%" source='https://towardsdatascience.com/from-a-single-decision-tree-to-a-random-forest-b9523be65147' align='center'/> </div>

## Using a random forest

So Random Forests are a bagged ensemble of Decision Trees. Just as a classifier, predictions are averaged in regression tasks, and voted in classification tasks.

Usually it's good to create many different trees (around 100), but not very deep.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_estimators=100,
                               #random_state=42
                              )

cross_val_score(forest, X, y, scoring = "r2", cv=5).mean()

### Bagging any algorithm

You can use bagging on many different algorithms, not just trees!

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=3)

knn_model.fit(X, y)
plot_decision_regions(X, y, clf=knn_model);

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

weak_learner = KNeighborsClassifier(n_neighbors=4)
bagged_model = BaggingClassifier(weak_learner, n_estimators=5)

bagged_model.fit(X, y)
plot_decision_regions(X, y, clf=bagged_model)

### Pros and Cons of Bagging

**Advantages**

- Reduces variance (overfitting)
- Can be applied to any model

**Disadvantages**

- Complex structure
- Long training time
- Disregards the **performance of individual sub-models**

## Boosting

Boosting is a method designed to train weak learners that learn from their predecessor's mistakes.

- It is a sequential ensemble method. So it cannot use several processors at the same time.
- The aim of boosting is to reduce bias.
- Focuses on the observations that are harder to predict.
- The best weak learners are given more weight in the final vote.

<div>
<img src="files/bagging_vs_boosting.png" width="75%" source='https://www.datacamp.com/tutorial/what-bagging-in-machine-learning-a-guide-with-examples' align='center'/> </div>

### Adaboost (Adaptative Boosting)

One of the most popular type of boosting is Adaboost. It can perform on several types of models.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(DecisionTreeClassifier(max_depth=3), # The default tree in AdaBoost, but with the hyperparameter max_depth
                               algorithm='SAMME',
                               n_estimators=50)  

model.fit(X, y)
plot_decision_regions(X, y, clf=model);

### Gradient Boosting

Gradient Boosting works only for trees, and is usually more performant than Adaboost. Instead of updating the weights of observations misclassified, it's going to predict for each next weak-learner what was misclassified by the previous model. So the trees become more and more complex with a lot of noise.

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators=5000, max_depth=3)    
model.fit(X, y)
plot_decision_regions(X, y, clf=model);

### XG Boost (Extreme Gradient Boosting)

This algorithm is is known for its high performance and efficiency compared to traditional gradient boosting implementations. It can be used using a different library than sklearn.

In [ ]:
%%time
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators=5000, max_depth=3)

model.fit(X, y)
plot_decision_regions(X, y, clf=model);

## Pros and Cons of Boosting

**Advantages**

- Strong sub-models have more influence in final decision.
- Reduces bias.

**Disadvantages**

- Computationally expensive (sequential).
- Easily overfits.
- Sensitive to outliers (too much time spent trying to correctly predict them).